In [11]:
# import library
import numpy as np
np.random.seed(678)

class Agent():
    
    def __init__(self,move_value):
        assert(move_value==1 or move_value==-1)
        self.move_value = move_value
        self.all_state = dict()
    
    def state(self,current_state):
        if str(current_state) in self.value:
            pass
        else: 
            self.value[str(current_state)] = 0

    def action(self,observation):
        
        if observation in self.all_state:
            action = self.all_state['next_move']
        return action

class Environment():
    
    def __init__(self):
        self.board = np.zeros((3,3))
    
    # Def: Check if there is any winner
    def check_winner(self):
        
        # ======== Check Winner By One Row ========         
        for row in self.board:
            if sum(row) == 3: return 1
            if sum(row) ==-3: return -1 
            
        for row in np.transpose(self.board):
            if sum(row) == 3: return 1
            if sum(row) ==-3: return -1 
        # ======== Check Winner By One Row ========         

                    
        # ======== Check Winner By Diag ========         
        if sum([self.board[i][i] for i in range(len(self.board))]) == 3:  return 1
        if sum([self.board[i][i] for i in range(len(self.board))]) == -3: return -1
        if sum([self.board[i][len(self.board)-i-1] for i in range(len(self.board))]) == 3: return  1       
        if sum([self.board[i][len(self.board)-i-1] for i in range(len(self.board))]) == -3:return -1  
        # ======== Check Winner By Diag ========         

        # ======== Check Draw ========         
        if abs(self.board).sum() == 9: return 0
        # ======== Check Draw ========         
            
    # Def: Check if there is a winner
    def check_if_there_is_winner(self): return self.check_winner()
    
    # change the borad position - location and value.      
    def make_a_move(self,position,value):
        assert (value == 1 or value == -1)
        assert (self.board.ravel()[position] == 0)
        self.board.ravel()[position] = value
    
    # Reset the board back the starting position.     
    def reset_game(self): self.board = np.zeros((3,3))
        
    # View the current state of the board
    def get_board(self): return self.board

In [12]:
# declare an agent and env
env = Environment()
agent1 = Agent(-1)
agent2 = Agent(1)

In [5]:
temp = Environment()
temp.make_a_move(0,-1)
temp.make_a_move(1,1)
temp.make_a_move(8,1)
print(temp.get_board())
print(temp.check_if_there_is_winner())
temp.make_a_move(2,-1)
temp.make_a_move(4,1)
temp.make_a_move(5,-1)
temp.make_a_move(3,1)
temp.make_a_move(7,-1)
temp.make_a_move(6,-1)
print(temp.get_board())
print(temp.check_if_there_is_winner())

[[-1.  1.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  1.]]
None
[[-1.  1. -1.]
 [ 1.  1. -1.]
 [-1. -1.  1.]]
0


In [115]:
temp_dict = dict()
temp_dict[str(temp.get_board())] = 4
temp_dict

{'[[-1.  1. -1.]\n [ 1.  1. -1.]\n [-1. -1.  1.]]': 4}

In [117]:
from collections import defaultdict
def tree(): return defaultdict(tree)
users = tree()
users['harold']['username'] = 'hrldcpr'
users['handler']['username'] = 'matthandlersux'

In [118]:
users

defaultdict(<function __main__.tree()>,
            {'harold': defaultdict(<function __main__.tree()>,
                         {'username': 'hrldcpr'}),
             'handler': defaultdict(<function __main__.tree()>,
                         {'username': 'matthandlersux'})})

In [119]:
! git all-go

[master 4ab60dc] commit
 2 files changed, 58 insertions(+), 76 deletions(-)
Counting objects: 4, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 858 bytes | 858.00 KiB/s, done.
Total 4 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/JaeDukSeo/tictactoe.git
   d0ab822..4ab60dc  master -> master
